In [1]:
# Install deps
%pip install "anthropic[vertex]"
# https://github.com/anthropics/anthropic-sdk-go

Looking in indexes: https://artifactory.gcp.anz/artifactory/api/pypi/pypi/simple/, https://artifactory.gcp.anz/artifactory/api/pypi/anzx-data-enablement/simple

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Users/elbanham/Developer/claude-gcp/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from os import environ

from anthropic import AnthropicVertex

region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = AnthropicVertex(region=region, project_id=project_id)
model = "claude-sonnet-4-5@20250929"

In [3]:
from typing import List, Optional


def add_user_message(messages: List[dict], content: str):
    _add_role_message(messages, "user", content)


def add_agent_message(messages: List[dict], content: str):
    _add_role_message(messages, "assistant", content)


def _add_role_message(messages: List[dict], role: str, content: str):
    messages.append({"role": role, "content": content})


""" Sends a chat request to the Claude model with the given messages. """


# Refer to https://platform.claude.com/docs/en/api/messages/create


def chat(
    messages: List[dict],
    system: Optional[str] = None,
    temperature: float = 1.0,
    stop_sequences: Optional[str] = None,
) -> str:
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system is not None:
        params["system"] = system

    response = client.messages.create(**params)
    return response.content[0].text

In [4]:
# Start with an empty message list
messages = []

# Add the initial user question
add_user_message(messages, "Define quantum computing in one sentence")

# Get Claude's response
answer = chat(messages)

# Add Claude's response to the conversation history
add_agent_message(messages, answer)

# Add a follow-up question
add_user_message(messages, "Write another sentence")

# Get the follow-up response with full context
final_answer = chat(messages)

final_answer

'Quantum computers use qubits instead of classical bits, allowing them to exist in multiple states simultaneously and perform many calculations at once.'

In [5]:
messages: List[dict] = []

while True:
    user_input = input("You: ")
    add_user_message(messages, user_input)
    response = chat(messages)
    add_agent_message(messages, response)
    print("----------------")
    print(response)
    print("----------------")

KeyboardInterrupt: Interrupted by user

In [6]:
messages: List[dict] = []

add_user_message(
    messages,
    "Write a python function that checks a string for duplicate chars",
)
answer = chat(
    messages,
    system="You are a software engineer. Provide only compact code snippets.",
    temperature=0.2,
)
print("Claude:", answer)

Claude: ```python
def has_duplicate_chars(s: str) -> bool:
    return len(s) != len(set(s))
```

**Usage:**
```python
has_duplicate_chars("hello")    # True (duplicate 'l')
has_duplicate_chars("world")    # False
has_duplicate_chars("aabbcc")   # True
```

**Alternative (returns the duplicate characters):**
```python
def find_duplicate_chars(s: str) -> set:
    seen = set()
    duplicates = set()
    for char in s:
        if char in seen:
            duplicates.add(char)
        seen.add(char)
    return duplicates
```


In [7]:
# Streaming example
messages: List[dict] = []
add_user_message(messages, "Explain the theory of relativity in simple terms.")
with client.messages.stream(model=model, max_tokens=1000, messages=messages) as stream:
    for text in stream.text_stream:
        pass
        print(text, end="", flush=True)  # Send to client in real application

final_message = stream.get_final_message()
print(f"Final message:{final_message}")

# Theory of Relativity - Simple Explanation

Einstein's theory has two parts:

## **Special Relativity (1905)**

**Main idea:** Space and time are connected, and nothing can travel faster than light.

Key points:
- **Time is relative** - A moving clock ticks slower than a stationary one. If you traveled in a spaceship near light speed, time would pass slower for you than for people on Earth.
- **Space is relative** - Moving objects appear shorter in their direction of travel.
- **E=mc²** - Energy and mass are interchangeable. A tiny bit of mass contains enormous energy.

## **General Relativity (1915)**

**Main idea:** Gravity isn't a force—it's the bending of space and time by massive objects.

Think of it like this:
- Imagine space as a stretched rubber sheet
- Heavy objects (like Earth or the Sun) create dips in the sheet
- Smaller objects roll toward these dips—that's what we experience as gravity

## **Real-world effects:**

- GPS satellites must account for relativity or they'd b

In [8]:
messages: List[dict] = []

add_user_message(messages, "Coffee or Tea ?")

## Example of message prefilling
add_agent_message(
    messages,
    "Coffee is better because",
)
answer = chat(messages)
print("Claude:", answer)

Claude:  it has more caffeine and a richer, bolder flavor that really wakes you up in the morning.

Actually, I should be more balanced here! It really depends on personal preference:

**Coffee** - Higher caffeine, bold flavor, great variety (espresso, cold brew, etc.)

**Tea** - Generally less caffeine, more subtle flavors, lots of health antioxidants, won't make you jittery

Both have their merits! What do you prefer, or are you trying to decide between them?


In [9]:
messages: List[dict] = []
add_user_message(messages, "Count from 1 to 10")
answer = chat(messages, stop_sequences=["5"])
answer

'1, 2, 3, 4, '

In [10]:
messages: List[dict] = []
add_user_message(
    messages,
    "Generate three different samples AWS cli commands. Each should be very short",
)
## Example of message prefilling
add_agent_message(
    messages,
    "Here are all commands in a single block with no cmments\n```bash",
)
answer = chat(messages, stop_sequences=["```"])
answer.strip()

'aws s3 ls\naws ec2 describe-instances\naws lambda list-functions'